In [1]:
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf


Descarga de datos

In [2]:
!wget https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0 -O books.csv
!wget https://www.dropbox.com/s/zpnnoy1i8ljf9fg/goodreads_bert_embeddings.npy?dl=0 -O goodreads_bert_embeddings.npy
!wget https://www.dropbox.com/s/a8hcc9w30y7r3jl/goodreads_bert_large_embeddings.npy?dl=0 -O goodreads_bert_large_embeddings.npy
!wget https://www.dropbox.com/s/dqeqpsr0vdvmcy0/goodreads_past_interactions.json?dl=0 -O goodreads_past_interactions.json
!wget https://www.dropbox.com/s/rjtzhmb2zbpp30q/goodreads_test_interactions.json?dl=0 -O goodreads_test_interactions.json

--2025-07-07 15:34:40--  https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/5s6xrfnu17yi34sfhmskb/books.csv?rlkey=ymzokbyqw3qq2bq5okfao9w1z&dl=0 [following]
--2025-07-07 15:34:40--  https://www.dropbox.com/scl/fi/5s6xrfnu17yi34sfhmskb/books.csv?rlkey=ymzokbyqw3qq2bq5okfao9w1z&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc65e808f68d87610411dfe29105.dl.dropboxusercontent.com/cd/0/inline/CtD-wVzThnpcnljw2TG0ozyCAeC7IsselUnTdU_LyvlLGP6j7BYIep12KgncefcyqcIzAxrwdsf-yzvf997mKGsTOcyK2kCs0j4E3fEoyTxAMqEgzoT0duAt5gmNmVNSuu-BBpkFxcKzFEFwdgOoE6g9/file# [following]
--2025-07-07 15:34:41--  https://uc65e808f68d87610411dfe29105.dl.dropboxusercontent.com/cd/0/in

In [3]:
df_books = pd.read_csv('books.csv')
df_books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,book_desc
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,Winning will make you famous. Losing means cer...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,Harry Potter's life is miserable. His parents ...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,About three things I was absolutely positive.F...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,The unforgettable novel of a childhood in a sl...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,Alternate Cover Edition ISBN: 0743273567 (ISBN...


In [4]:
with open('goodreads_past_interactions.json', 'r') as f:
    user_interactions = json.load(f)


In [5]:
with open('goodreads_test_interactions.json', 'r') as f:
    user_interactions_test = json.load(f)

In [6]:
idx2userid = {i: id_ for i, id_ in enumerate(user_interactions.keys())}
userid2idx = {id_:i for i, id_ in enumerate(user_interactions.keys())}

In [7]:
# 2) Build userid2idx by enumerating all distinct user IDs in the training set.
#    (You could also include users from test if there are any users in test not in train,
#     but typically we drop test-users that never appear in train.)

userid2idx = { raw_uid: idx
               for idx, raw_uid in enumerate(user_interactions.keys()) }

#    If you want to ensure we include any “new” users that appear only in test:
for raw_uid in user_interactions_test.keys():
    if raw_uid not in userid2idx:
        userid2idx[raw_uid] = len(userid2idx)

In [8]:
# 3) Build the set of all book IDs that appear in either train or test:
all_book_ids = set()

#    From train:
for raw_uid, book_list in user_interactions.items():
    for raw_bid in book_list:
        all_book_ids.add(raw_bid)

#    From test:
for raw_uid, book_list in user_interactions_test.items():
    for raw_bid in book_list:
        all_book_ids.add(raw_bid)

In [9]:
# 4) Now enumerate that set to build bookid2idx
bookid2idx = { raw_bid: idx for idx, raw_bid in enumerate(all_book_ids) }

In [10]:
# 5) Now you can check:
num_users = len(userid2idx)
num_items = len(bookid2idx)

print(f"Number of distinct users: {num_users}")
print(f"Number of distinct books: {num_items}")

Number of distinct users: 52821
Number of distinct books: 4287


In [11]:
unique_books = df_books.shape[0]
num_columns_table = df_books.shape[1]

print(f"Number of unique books: {unique_books}")
print(f"Number of columns in the table: {num_columns_table}")

Number of unique books: 4287
Number of columns in the table: 24


## Build “(user, item, label)” arrays for training

First, we’ll construct a dictionary of “positive item-indices per user_idx” for training:

In [12]:
from collections import defaultdict

# pos_items_per_user[u_idx] = set of i_idx that user u_idx interacted with in train
pos_items_per_user = defaultdict(set)

for raw_u, book_list in user_interactions.items():
    u_idx = userid2idx[raw_u]
    for raw_b in book_list:
        i_idx = bookid2idx[raw_b]
        pos_items_per_user[u_idx].add(i_idx)

# (Now pos_items_per_user[u_idx] is a Python set of i_idx for each user)

Next, negative sampling. For each positive pair (u_idx, i_idx), we will sample n_neg_per_pos “negatives” j ∈ [0..num_items−1] such that j ∉ pos_items_per_user[u_idx]. Let’s choose 4 negatives per positive:

In [13]:
n_neg_per_pos = 2
rng = np.random.default_rng(seed=42)

user_input = []
item_input = []
labels = []

for u_idx, pos_set in pos_items_per_user.items():
    # If a user has no positives (unlikely), skip.
    if len(pos_set) == 0:
        continue

    for i_idx in pos_set:
        # — Positive sample —
        user_input.append(u_idx)
        item_input.append(i_idx)
        labels.append(1.0)

        # — Negative samples —
        neg_count = 0
        while neg_count < n_neg_per_pos:
            sampled_i = rng.integers(low=0, high=num_items)
            if sampled_i not in pos_set:
                user_input.append(u_idx)
                item_input.append(sampled_i)
                labels.append(0.0)
                neg_count += 1

# Convert lists → NumPy arrays for Keras
user_input = np.array(user_input, dtype=np.int32)   # shape = (N_total_examples,)
item_input = np.array(item_input, dtype=np.int32)   # shape = (N_total_examples,)
labels     = np.array(labels,     dtype=np.float32) # shape = (N_total_examples,)

print("train triples:", user_input.shape, item_input.shape, labels.shape)


train triples: (10063569,) (10063569,) (10063569,)


## Build (user, item, label) arrays for validation (test)

In [14]:
# 1) Construct pos_items_per_user_test[u_idx]
pos_items_per_user_test = defaultdict(set)
for raw_u, book_list in user_interactions_test.items():
    # It’s possible that user_interactions_test contains a user not seen in training.
    # If so, skip them (or add them to userid2idx + pos_items_per_user if you want cold-start eval).
    if raw_u not in userid2idx:
        continue
    u_idx = userid2idx[raw_u]
    for raw_b in book_list:
        if raw_b not in bookid2idx:
            # skip books that weren’t in your mapping
            continue
        i_idx = bookid2idx[raw_b]
        pos_items_per_user_test[u_idx].add(i_idx)

# 2) Negative sampling for test:
user_input_test = []
item_input_test = []
labels_test = []

for u_idx, pos_set in pos_items_per_user_test.items():
    if len(pos_set) == 0:
        continue

    for i_idx in pos_set:
        # Positive
        user_input_test.append(u_idx)
        item_input_test.append(i_idx)
        labels_test.append(1.0)

        # Negatives
        neg_count = 0
        while neg_count < n_neg_per_pos:
            sampled_i = rng.integers(low=0, high=num_items)
            if sampled_i not in pos_set:
                user_input_test.append(u_idx)
                item_input_test.append(sampled_i)
                labels_test.append(0.0)
                neg_count += 1

user_input_test = np.array(user_input_test, dtype=np.int32)
item_input_test = np.array(item_input_test, dtype=np.int32)
labels_test     = np.array(labels_test,     dtype=np.float32)

print("test triples:", user_input_test.shape, item_input_test.shape, labels_test.shape)

test triples: (3000,) (3000,) (3000,)


## Item features

In [15]:
bert_embeddings = np.load("goodreads_bert_embeddings.npy")  # shape: (num_items, embedding_dim)
#bert_embeddings_large = np.load("goodreads_bert_large_embeddings.npy")

num_items, embedding_dim = bert_embeddings.shape

print(bert_embeddings.shape)


(4287, 768)


In [16]:
# For training
#item_feat_input = item_features[item_input]         # shape = (N_train, D)
#item_feat_input = bert_embeddings[item_input]
# For validation
#item_feat_input_test = item_features[item_input_test]
#item_feat_input_test = bert_embeddings[item_input_test]

# Functions for Ranking Metrics

## Metricas de precision

In [17]:
import numpy as np

def precision_at_k(predicted_items, true_set, k):
    """
    predicted_items: list or 1D array of item-indices ranked by descending score (length ≥ k)
    true_set: a Python set of ground-truth positive items for this user
    k: cutoff
    """
    pred_k = predicted_items[:k]
    hits = sum((1 for i in pred_k if i in true_set))
    return hits / k

def dcg_at_k(predicted_items, true_set, k):
    """
    DCG@k = sum_{i=1..k} ( rel_i / log2(i+1) ), where rel_i = 1 if predicted_items[i-1] ∈ true_set
    """
    dcg = 0.0
    for rank, item in enumerate(predicted_items[:k], start=1):
        if item in true_set:
            dcg += 1.0 / np.log2(rank + 1)
    return dcg

def idcg_at_k(true_set, k):
    """
    Ideal DCG@k for this user, i.e. if we ranked all true items first.
    IDCG = sum_{i=1..min(len(true_set), k)} (1 / log2(i+1))
    """
    n_rel = min(len(true_set), k)
    if n_rel == 0:
        return 0.0
    return sum((1.0 / np.log2(i + 1) for i in range(1, n_rel + 1)))

def ndcg_at_k(predicted_items, true_set, k):
    """
    NDCG@k = DCG@k / IDCG@k
    """
    dcg = dcg_at_k(predicted_items, true_set, k)
    idcg = idcg_at_k(true_set, k)
    return 0.0 if idcg == 0 else dcg / idcg

def average_precision_at_k(predicted_items, true_set, k):
    """
    AP@k for a single user:
    AP = (1 / min(n_pos, k)) * sum_{i=1..k} ( precision@i * rel_i ),
    where rel_i = 1 if predicted_items[i-1] ∈ true_set, else 0.
    If the user has no positives, define AP = 0.0.
    """
    if len(true_set) == 0:
        return 0.0

    hits = 0
    score = 0.0
    for idx, item in enumerate(predicted_items[:k], start=1):
        if item in true_set:
            hits += 1
            score += hits / idx  # precision@idx
    return score / min(len(true_set), k)


Prepare per‐user ground‐truth sets

In [18]:
from collections import defaultdict

# 1) Build train_pos (you probably have this as pos_items_per_user):
train_pos = defaultdict(set)
for raw_u, book_list in user_interactions.items():
    u_idx = userid2idx[raw_u]
    for raw_b in book_list:
        train_pos[u_idx].add(bookid2idx[raw_b])

# 2) Build test_pos similarly:
test_pos = defaultdict(set)
for raw_u, book_list in user_interactions_test.items():
    if raw_u not in userid2idx:
        continue
    u_idx = userid2idx[raw_u]
    for raw_b in book_list:
        if raw_b not in bookid2idx:
            continue
        test_pos[u_idx].add(bookid2idx[raw_b])


In [19]:
all_users = list(test_pos.keys())  # only users with ≥1 test interaction
num_items_check = len(bookid2idx)        # total number of distinct items


## Metricas de diversidad y novelty

Contador de Popularidad en Train

In [20]:
from collections import Counter
all_train_pairs = [
    (u_idx, i_idx)
    for u_idx, books in user_interactions.items()
    for i_idx in [bookid2idx[b] for b in books]
]
pop_counter = Counter(i for _, i in all_train_pairs)
max_pop = max(pop_counter.values())

Funciones

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

# Intra list diversity
def ild_at_k(topk, feat_matrix, k):
    vecs = feat_matrix[topk[:k]]              # (k, D)
    sims = cosine_similarity(vecs)            # (k, k)
    i, j = np.triu_indices(k, k=1)
    return np.mean(1.0 - sims[i, j])

# Novelty basado en popularidad global
def novelty_popularity(topk, pop_counter, max_pop, k):
    pops = np.array([pop_counter[i] for i in topk[:k]], dtype=np.float32)
    return np.mean(1.0 - pops / max_pop)

## Custom Keras callback to compute metrics

In [22]:
import tensorflow as tf

class RankingMetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_pos, test_pos, num_items, k=10, verbose=1):
        """
        train_pos: dict[u_idx] -> set of item indices the user interacted with in training
        test_pos: dict[u_idx] -> set of item indices for ground-truth test positives
        num_items: total number of distinct items
        k: cutoff for precision@k, NDCG@k, MAP@k
        verbose: 0 or 1 (whether to print results)
        """
        super().__init__()
        self.train_pos = train_pos
        self.test_pos = test_pos
        self.num_items = num_items
        self.k = k
        self.verbose = verbose

        # Precompute for speed: a full array of all item indices
        self.all_items_arr = np.arange(num_items, dtype=np.int32)

    def on_epoch_end(self, epoch, logs=None):
        """
        At the end of each epoch, compute average Precision@k, NDCG@k, MAP@k over all users
        in test_pos. Adds metrics to logs dictionary and prints them.
        """
        precision_list = []
        ndcg_list = []
        ap_list = []
        ilds = []
        novelties = []

        # For each user in test_pos:
        for u_idx, true_set in self.test_pos.items():
            if not true_set:
                continue

            # 1) Build candidate items = all_items \ train_pos[u]
            train_set = self.train_pos.get(u_idx, set())
            # Boolean mask for candidates:
            if len(train_set) == 0:
                candidates = self.all_items_arr
            else:
                mask = np.ones(self.num_items, dtype=bool)
                mask[list(train_set)] = False
                candidates = self.all_items_arr[mask]

            # 2) Build user and item batches for prediction
            u_batch = np.full(shape=(candidates.shape[0],), fill_value=u_idx, dtype=np.int32)
            i_batch = candidates

            # 3) Get scores from the model
            #    Note: model.predict(...) can be batched inside (it handles large arrays)
            scores = self.model.predict([u_batch, i_batch], batch_size=2048, verbose=0).flatten()

            # 4) Get top-k items by descending score
            topk_indices = np.argpartition(-scores, self.k - 1)[:self.k]
            # `argpartition` is O(n), then we sort those k for final ranking:
            topk_scores = scores[topk_indices]
            topk_items = candidates[topk_indices][np.argsort(-topk_scores)]

            # 5) Compute metrics for this user
            p_at_k = precision_at_k(topk_items, true_set, self.k)
            ndcg_k = ndcg_at_k(topk_items, true_set, self.k)
            ap_k = average_precision_at_k(topk_items, true_set, self.k)

            ild_k = ild_at_k(topk_items, bert_embeddings, self.k)
            novelty_k = novelty_popularity(topk_items, pop_counter, max_pop, self.k)

            precision_list.append(p_at_k)
            ndcg_list.append(ndcg_k)
            ap_list.append(ap_k)

            ilds.append(ild_k)
            novelties.append(novelty_k)

        # 6) Average across all users
        avg_precision = float(np.mean(precision_list)) if precision_list else 0.0
        avg_ndcg      = float(np.mean(ndcg_list))      if ndcg_list else 0.0
        avg_map       = float(np.mean(ap_list))        if ap_list else 0.0
        avg_ild       = float(np.mean(ilds))          if ilds else 0.0
        avg_novelty   = float(np.mean(novelties))     if novelties else 0.0

        # 7) Log into `logs` so Keras will record them
        if logs is not None:
            logs[f"Precision@{self.k}"] = avg_precision
            logs[f"NDCG@{self.k}"]      = avg_ndcg
            logs[f"MAP@{self.k}"]       = avg_map
            logs[f"ILD@{self.k}"]      = avg_ild
            logs[f"Novelty@{self.k}"]   = avg_novelty

        # 8) Optionally print to console
        if self.verbose > 0:
            print(
                f" — epoch {epoch + 1:>2d}: "
                f"Precision@{self.k}={avg_precision:.4f}, "
                f"NDCG@{self.k}={avg_ndcg:.4f}, "
                f"MAP@{self.k}={avg_map:.4f}"
                f"ILD@{self.k}={avg_ild:.4f}"
                f"Novelty@{self.k}={avg_novelty:.4f}"
            )


In [23]:
import tensorflow as tf

class MultiKRankingCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_pos, test_pos, num_items, Ks=[5,10,20], verbose=1):
        """
        train_pos: dict[u_idx] -> set of item indices the user interacted with in training
        test_pos: dict[u_idx] -> set of item indices for ground-truth test positives
        num_items: total number of distinct items
        k: cutoff for precision@k, NDCG@k, MAP@k
        verbose: 0 or 1 (whether to print results)
        """
        super().__init__()
        self.train_pos = train_pos
        self.test_pos = test_pos
        self.num_items = num_items
        self.Ks = Ks
        self.verbose = verbose

        # Precompute for speed: a full array of all item indices
        self.all_items_arr = np.arange(num_items, dtype=np.int32)

    def on_epoch_end(self, epoch, logs=None):
        """
        At the end of each epoch, compute average Precision@k, NDCG@k, MAP@k over all users
        in test_pos. Adds metrics to logs dictionary and prints them.
        """
        precision_list = []
        ndcg_list = []
        ap_list = []
        ilds = []
        novelties = []

        metrics = {K: {"prec":[], "ndcg":[], "map":[],"ild":[], "nov":[]} for K in self.Ks}

        # For each user in test_pos:
        for u_idx, true_set in self.test_pos.items():
            if not true_set:
                continue

            # 1) Build candidate items = all_items \ train_pos[u]
            train_set = self.train_pos.get(u_idx, set())
            # Boolean mask for candidates:
            if len(train_set) == 0:
                candidates = self.all_items_arr
            else:
                mask = np.ones(self.num_items, dtype=bool)
                mask[list(train_set)] = False
                candidates = self.all_items_arr[mask]

            # 2) Build user and item batches for prediction
            u_batch = np.full(shape=(candidates.shape[0],), fill_value=u_idx, dtype=np.int32)
            i_batch = candidates

            # 3) Get scores from the model
            #    Note: model.predict(...) can be batched inside (it handles large arrays)
            scores = self.model.predict([u_batch, i_batch], batch_size=2048, verbose=0).flatten()

            for K in self.Ks:
                # 4) Get top-k items by descending score
                topk_indices = np.argpartition(-scores, K - 1)[:K]
                # `argpartition` is O(n), then we sort those k for final ranking:
                topk_scores = scores[topk_indices]
                topk_items = candidates[topk_indices][np.argsort(-topk_scores)]

                # 5) Compute metrics for this user
                p_at_k = precision_at_k(topk_items, true_set, K)
                ndcg_k = ndcg_at_k(topk_items, true_set, K)
                ap_k = average_precision_at_k(topk_items, true_set, K)

                ild_k = ild_at_k(topk_items, bert_embeddings, K)
                novelty_k = novelty_popularity(topk_items, pop_counter, max_pop, K)

                m = metrics[K]
                m["prec"].append(p_at_k)
                m["ndcg"].append(ndcg_k)
                m["map"].append(ap_k)
                m["ild"].append(ild_k)
                m["nov"].append(novelty_k)

        # 6) Average across all users and log so Keras will record them
        for K in self.Ks:
            m = metrics[K]
            logs[f"Precision@{K}"] = float(np.mean(m["prec"])) if m["prec"] else 0.0
            logs[f"NDCG@{K}"]      = float(np.mean(m["ndcg"])) if m["ndcg"] else 0.0
            logs[f"MAP@{K}"]       = float(np.mean(m["map"])) if m["map"] else 0.0
            logs[f"ILD@{K}"]      = float(np.mean(m["ild"])) if m["ild"] else 0.0
            logs[f"Novelty@{K}"]   = float(np.mean(m["nov"])) if m["nov"] else 0.0

        # 8) Optionally print to console
        if self.verbose > 0:
            line = f" — epoch {epoch + 1}: "
            for K in self.Ks:
                m = metrics[K]
                line += (
                    f"  P@{K}={logs[f'Precision@{K}']:.3f},"
                    f" NDCG@{K}={logs[f'NDCG@{K}']:.3f},"
                    f" MAP@{K}={logs[f'MAP@{K}']:.3f},"
                    f" ILD@{K}={logs[f'ILD@{K}']:.3f},"
                    f" Nov@{K}={logs[f'Novelty@{K}']:.3f};"
                )
            print(line)


# Vanilla NCF (Keras)


In [24]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Multiply, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

# 1) Reduced hyperparameters
K_mf  = 16
K_mlp = 32
mlp_layers = [32, 16]
dropout_rate = 0.2

# 2) Define inputs
user_in = Input(shape=(), dtype=tf.int32, name="user_input")
item_in = Input(shape=(), dtype=tf.int32, name="item_input")

# 3) GMF branch
u_mf = Embedding(num_users, K_mf, name="u_embed_mf")(user_in)
i_mf = Embedding(num_items, K_mf, name="i_embed_mf")(item_in)
u_latent_mf = Flatten()(u_mf)
i_latent_mf = Flatten()(i_mf)
mf_vec = Multiply()([u_latent_mf, i_latent_mf])  # (batch, K_mf)

# 4) MLP branch (smaller dims)
u_mlp_emb = Embedding(num_users, K_mlp, name="u_embed_mlp")(user_in)
i_mlp_emb = Embedding(num_items, K_mlp, name="i_embed_mlp")(item_in)
u_latent_mlp = Flatten()(u_mlp_emb)
i_latent_mlp = Flatten()(i_mlp_emb)

mlp_vec = Concatenate()([u_latent_mlp, i_latent_mlp])  # (batch, 2*K_mlp)
for idx, size in enumerate(mlp_layers):
    mlp_vec = Dense(units=size, activation="relu", name=f"mlp_dense_{idx}")(mlp_vec)
    mlp_vec = Dropout(dropout_rate)(mlp_vec)

# 5) Combine GMF + MLP
concat = Concatenate()([mf_vec, mlp_vec])
output = Dense(units=1, activation="sigmoid", name="prediction")(concat)

model_vanilla = Model(inputs=[user_in, item_in], outputs=output)
model_vanilla.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name="auc")]
)
model_vanilla.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mlp         │ (None, 32)        │  1,690,272 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mlp         │ (None, 32)        │    137,184 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 32)        │          0 │ u_embed_mlp[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 32)        │          0 │ i_embed_mlp[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64)        │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_0 (Dense) │ (None, 32)        │      2,080 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mf          │ (None, 16)        │    845,136 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mf          │ (None, 16)        │     68,592 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32)        │          0 │ mlp_dense_0[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 16)        │          0 │ u_embed_mf[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 16)        │          0 │ i_embed_mf[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_1 (Dense) │ (None, 16)        │        528 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 16)        │          0 │ flatten[0][0],    │
│                     │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 16)        │          0 │ mlp_dense_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 32)        │          0 │ multiply[0][0],   │
│ (Concatenate)       │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prediction (Dense)  │ (None, 1)         │         33 │ concatenate_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,743,825 (10.47 MB)

 Trainable params: 2,743,825 (10.47 MB)

 Non-trainable params: 0 (0.00 B)

Before training, prepare callback for metrics

In [25]:
ks = [5, 10, 20]
rank_cb = MultiKRankingCallback(
    train_pos=train_pos,      # user → set of train-positive items
    test_pos=test_pos,        # user → set of test-positive items
    num_items=num_items,      # total number of distinct items (4287)
    Ks=ks,                     # compute Precision@10, NDCG@10, MAP@10
    verbose=1                 # set to 1 to print metrics each epoch
)

Train on tf.data.Dataset


In [26]:
batch_size = 4096  # increase if your GPU/Colab can handle it (maybe 4096)
buffer_size = 100_000

# 1) Build a Dataset from your three 1D NumPy arrays
#    Each element is (u_idx, i_idx, label).
train_ds = tf.data.Dataset.from_tensor_slices((user_input, item_input, labels))

# 2) Shuffle → batch → prefetch.  Drop remainder so each batch is exactly `batch_size`.
train_ds = (
    train_ds
    .shuffle(buffer_size, seed=42)
    .batch(batch_size, drop_remainder=True)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 3) Do the same for validation (but typically you skip shuffle for val):
val_ds = tf.data.Dataset.from_tensor_slices(
    (user_input_test, item_input_test, labels_test)
)
val_ds = (
    val_ds
    .batch(batch_size, drop_remainder=False)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 4) Now call .fit on the Datasets:
history_vanilla = model_vanilla.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
    callbacks=[rank_cb],
    shuffle=False  # we already shuffled in the Dataset pipeline
)


Epoch 1/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - auc: 0.8236 - loss: 0.4785 — epoch 1:   P@5=0.108, NDCG@5=0.119, MAP@5=0.069, ILD@5=0.312, Nov@5=0.496;  P@10=0.092, NDCG@10=0.103, MAP@10=0.043, ILD@10=0.311, Nov@10=0.555;  P@20=0.087, NDCG@20=0.148, MAP@20=0.057, ILD@20=0.304, Nov@20=0.628;
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 180s 72ms/step - auc: 0.8236 - loss: 0.4785 - val_auc: 0.9182 - val_loss: 0.3514 - Precision@5: 0.1080 - NDCG@5: 0.1190 - MAP@5: 0.0686 - ILD@5: 0.3123 - Novelty@5: 0.4963 - Precision@10: 0.0920 - NDCG@10: 0.1027 - MAP@10: 0.0426 - ILD@10: 0.3108 - Novelty@10: 0.5548 - Precision@20: 0.0865 - NDCG@20: 0.1485 - MAP@20: 0.0568 - ILD@20: 0.3040 - Novelty@20: 0.6282
Epoch 2/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - auc: 0.9094 - loss: 0.3570 — epoch 2:   P@5=0.122, NDCG@5=0.133, MAP@5=0.075, ILD@5=0.308, Nov@5=0.545;  P@10=0.109, NDCG@10=0.119, MAP@10=0.049, ILD@10=0.307, Nov@10=0.600;  P@20=0.096, NDCG@20=0.166, MAP@20=0.064, ILD@20=0.304, Nov@20=0.661;
2456/

# NCF + Extra Embeddings

In [52]:
import numpy as np

# Suppose bert_embeddings is a NumPy array of shape (num_items, D)
bert_embeddings = np.load("goodreads_bert_embeddings.npy")  # (num_items, D)
num_items, D = bert_embeddings.shape

# Create a Keras Embedding layer that simply returns bert_embeddings[i]
bert_embed_layer = tf.keras.layers.Embedding(
    input_dim=num_items,
    output_dim=D,
    embeddings_initializer=tf.keras.initializers.Constant(bert_embeddings),
    trainable=False,          # freeze so weights never change
    name="bert_embed_lookup"
)


In [53]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Multiply, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy("mixed_float16")

K_mf   = 16
K_mlp  = 32
mlp_layers = [64, 32]   # just two hidden layers
dropout_rate = 0.2

# --- Model Definition ---
# 1) Define just integer inputs
user_in = Input(shape=(), dtype=tf.int32, name="user_input")
item_in = Input(shape=(), dtype=tf.int32, name="item_input")

# 2) GMF branch (unchanged)
u_mf = Embedding(num_users, K_mf, name="u_embed_mf")(user_in)
i_mf = Embedding(num_items, K_mf, name="i_embed_mf")(item_in)
u_latent_mf = Flatten()(u_mf)
i_latent_mf = Flatten()(i_mf)
mf_vec = Multiply()([u_latent_mf, i_latent_mf])  # (batch, K_mf)

# 3) MLP branch: learned user/item embeddings
u_mlp = Embedding(num_users, K_mlp, name="u_embed_mlp")(user_in)
i_mlp = Embedding(num_items, K_mlp, name="i_embed_mlp")(item_in)
u_latent_mlp = Flatten()(u_mlp)
i_latent_mlp = Flatten()(i_mlp)

# 4) **Here is where we “lookup” BERT features** for each item index
bert_feat = bert_embed_layer(item_in)  # shape = (batch_size, D)

# 5) Concatenate (u_latent_mlp, i_latent_mlp, bert_feat)
merged_mlp = Concatenate(name="mlp_input_concat")(
    [u_latent_mlp, i_latent_mlp, bert_feat]
)  # shape = (batch_size, 2*K_mlp + D)

# 6) Continue with your MLP layers
x = merged_mlp
for idx, hidden_size in enumerate(mlp_layers):
    x = Dense(hidden_size, activation="relu", name=f"mlp_dense_{idx}")(x)
    x = Dropout(dropout_rate)(x)

# 7) Final “fusion” with GMF part
final_concat = Concatenate()([mf_vec, x])
output = Dense(1, activation="sigmoid", name="prediction")(final_concat)

model_with_feats = Model(inputs=[user_in, item_in], outputs=output)
model_with_feats.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name="auc")]
)
model_with_feats.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mlp         │ (None, 32)        │  1,690,272 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mlp         │ (None, 32)        │    137,184 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_6 (Flatten) │ (None, 32)        │          0 │ u_embed_mlp[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_7 (Flatten) │ (None, 32)        │          0 │ i_embed_mlp[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_embed_lookup   │ (None, 768)       │  3,292,416 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input_concat    │ (None, 832)       │          0 │ flatten_6[0][0],  │
│ (Concatenate)       │                   │            │ flatten_7[0][0],  │
│                     │                   │            │ bert_embed_looku… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_0 (Dense) │ (None, 64)        │     53,312 │ mlp_input_concat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mf          │ (None, 16)        │    845,136 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mf          │ (None, 16)        │     68,592 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ mlp_dense_0[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 16)        │          0 │ u_embed_mf[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_5 (Flatten) │ (None, 16)        │          0 │ i_embed_mf[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_1 (Dense) │ (None, 32)        │      2,080 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 16)        │          0 │ flatten_4[0][0],  │
│ (Multiply)          │                   │            │ flatten_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 32)        │          0 │ mlp_dense_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 48)        │          0 │ multiply_1[0][0], │
│ (Concatenate)       │                   │            │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prediction (Dense)  │ (None, 1)         │         49 │ concatenate_2[0]… │
└─────────────────────┴───────────────────┴────────────┴─────────────────

 Total params: 6,089,041 (23.23 MB)

 Trainable params: 2,796,625 (10.67 MB)

 Non-trainable params: 3,292,416 (12.56 MB)

In [54]:
ks = [5, 10, 20]
rank_cb = MultiKRankingCallback(
    train_pos=train_pos,      # user → set of train-positive items
    test_pos=test_pos,        # user → set of test-positive items
    num_items=num_items,      # total number of distinct items (4287)
    Ks=ks,                     # compute Precision@10, NDCG@10, MAP@10
    verbose=1                 # set to 1 to print metrics each epoch
)

Train with extra embeddings

In [55]:
batch_size = 4096  # increase if your GPU/Colab can handle it (maybe 4096)
buffer_size = 100_000

# 1) Build a Dataset from your three 1D NumPy arrays
#    Each element is (u_idx, i_idx, label).
train_ds = tf.data.Dataset.from_tensor_slices((user_input, item_input, labels))

# 2) Shuffle → batch → prefetch.  Drop remainder so each batch is exactly `batch_size`.
train_ds = (
    train_ds
    .shuffle(buffer_size, seed=42)
    .batch(batch_size, drop_remainder=True)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 3) Do the same for validation (but typically you skip shuffle for val):
val_ds = tf.data.Dataset.from_tensor_slices(
    (user_input_test, item_input_test, labels_test)
)
val_ds = (
    val_ds
    .batch(batch_size, drop_remainder=False)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 4) Now call .fit on the Datasets:
history_with_feats = model_with_feats.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
    callbacks=[rank_cb],
    shuffle=False  # we already shuffled in the Dataset pipeline
)


Epoch 1/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - auc: 0.8337 - loss: 0.4592 — epoch 1:   P@5=0.100, NDCG@5=0.116, MAP@5=0.064, ILD@5=0.320, Nov@5=0.499;  P@10=0.086, NDCG@10=0.100, MAP@10=0.041, ILD@10=0.312, Nov@10=0.543;  P@20=0.084, NDCG@20=0.145, MAP@20=0.054, ILD@20=0.303, Nov@20=0.613;
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 326s 131ms/step - auc: 0.8337 - loss: 0.4592 - val_auc: 0.9125 - val_loss: 0.3572 - Precision@5: 0.1000 - NDCG@5: 0.1159 - MAP@5: 0.0642 - ILD@5: 0.3202 - Novelty@5: 0.4994 - Precision@10: 0.0860 - NDCG@10: 0.1000 - MAP@10: 0.0406 - ILD@10: 0.3117 - Novelty@10: 0.5434 - Precision@20: 0.0835 - NDCG@20: 0.1454 - MAP@20: 0.0538 - ILD@20: 0.3035 - Novelty@20: 0.6128
Epoch 2/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - auc: 0.9099 - loss: 0.3556 — epoch 2:   P@5=0.102, NDCG@5=0.110, MAP@5=0.059, ILD@5=0.306, Nov@5=0.548;  P@10=0.105, NDCG@10=0.110, MAP@10=0.043, ILD@10=0.303, Nov@10=0.592;  P@20=0.098, NDCG@20=0.162, MAP@20=0.059, ILD@20=0.306, Nov@20=0.646;
24

# NCF + ResNet images

In [56]:
portrait_embeddings = np.load("resnet_book_embeddings.npy")  # (num_items, D)
num_items, D2 = portrait_embeddings.shape

portrait_embeddings.shape
print(f"num_items={num_items}, D={D2}")


num_items=4287, D=2048


In [57]:
import numpy as np

# Suppose portrait_embeddings is a NumPy array of shape (num_items, D)
portrait_embeddings = np.load("resnet_book_embeddings.npy")  # (num_items, D)
num_items, D = portrait_embeddings.shape

# Create a Keras Embedding layer that simply returns bert_embeddings[i]
resnet_embed_layer = tf.keras.layers.Embedding(
    input_dim=num_items,
    output_dim=D,
    embeddings_initializer=tf.keras.initializers.Constant(portrait_embeddings),
    trainable=False,          # freeze so weights never change
    name="resnet_embed_lookup"
)


In [58]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Multiply, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy("mixed_float16")

K_mf   = 16
K_mlp  = 32
mlp_layers = [64, 32]   # just two hidden layers
dropout_rate = 0.2

# --- Model Definition ---
# 1) Define just integer inputs
user_in = Input(shape=(), dtype=tf.int32, name="user_input")
item_in = Input(shape=(), dtype=tf.int32, name="item_input")

# 2) GMF branch (unchanged)
u_mf = Embedding(num_users, K_mf, name="u_embed_mf")(user_in)
i_mf = Embedding(num_items, K_mf, name="i_embed_mf")(item_in)
u_latent_mf = Flatten()(u_mf)
i_latent_mf = Flatten()(i_mf)
mf_vec = Multiply()([u_latent_mf, i_latent_mf])  # (batch, K_mf)

# 3) MLP branch: learned user/item embeddings
u_mlp = Embedding(num_users, K_mlp, name="u_embed_mlp")(user_in)
i_mlp = Embedding(num_items, K_mlp, name="i_embed_mlp")(item_in)
u_latent_mlp = Flatten()(u_mlp)
i_latent_mlp = Flatten()(i_mlp)

# 4) **Here is where we “lookup” resnet features** for each item index
resnet_feat = resnet_embed_layer(item_in)  # shape = (batch_size, D)

# 5) Concatenate (u_latent_mlp, i_latent_mlp, bert_feat)
merged_mlp = Concatenate(name="mlp_input_concat")(
    [u_latent_mlp, i_latent_mlp, resnet_feat]
)  # shape = (batch_size, 2*K_mlp + D)

# 6) Continue with your MLP layers
x = merged_mlp
for idx, hidden_size in enumerate(mlp_layers):
    x = Dense(hidden_size, activation="relu", name=f"mlp_dense_{idx}")(x)
    x = Dropout(dropout_rate)(x)

# 7) Final “fusion” with GMF part
final_concat = Concatenate()([mf_vec, x])
output = Dense(1, activation="sigmoid", name="prediction")(final_concat)

model_with_feats_2 = Model(inputs=[user_in, item_in], outputs=output)
model_with_feats_2.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name="auc")]
)
model_with_feats_2.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mlp         │ (None, 32)        │  1,690,272 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mlp         │ (None, 32)        │    137,184 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_10          │ (None, 32)        │          0 │ u_embed_mlp[0][0] │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_11          │ (None, 32)        │          0 │ i_embed_mlp[0][0] │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet_embed_lookup │ (None, 2048)      │  8,779,776 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input_concat    │ (None, 2112)      │          0 │ flatten_10[0][0], │
│ (Concatenate)       │                   │            │ flatten_11[0][0], │
│                     │                   │            │ resnet_embed_loo… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_0 (Dense) │ (None, 64)        │    135,232 │ mlp_input_concat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mf          │ (None, 16)        │    845,136 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mf          │ (None, 16)        │     68,592 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 64)        │          0 │ mlp_dense_0[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_8 (Flatten) │ (None, 16)        │          0 │ u_embed_mf[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_9 (Flatten) │ (None, 16)        │          0 │ i_embed_mf[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_1 (Dense) │ (None, 32)        │      2,080 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_2          │ (None, 16)        │          0 │ flatten_8[0][0],  │
│ (Multiply)          │                   │            │ flatten_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 32)        │          0 │ mlp_dense_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 48)        │          0 │ multiply_2[0][0], │
│ (Concatenate)       │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 11,658,321 (44.47 MB)

 Trainable params: 2,878,545 (10.98 MB)

 Non-trainable params: 8,779,776 (33.49 MB)

Training

In [59]:
ks = [5, 10, 20]
rank_cb = MultiKRankingCallback(
    train_pos=train_pos,      # user → set of train-positive items
    test_pos=test_pos,        # user → set of test-positive items
    num_items=num_items,      # total number of distinct items (4287)
    Ks=ks,                     # compute Precision@k, NDCG@k, MAP@k
    verbose=1                 # set to 1 to print metrics each epoch
)

In [60]:
batch_size = 4096  # increase if your GPU/Colab can handle it (maybe 4096)
buffer_size = 100_000

# 1) Build a Dataset from your three 1D NumPy arrays
#    Each element is (u_idx, i_idx, label).
train_ds = tf.data.Dataset.from_tensor_slices((user_input, item_input, labels))

# 2) Shuffle → batch → prefetch.  Drop remainder so each batch is exactly `batch_size`.
train_ds = (
    train_ds
    .shuffle(buffer_size, seed=42)
    .batch(batch_size, drop_remainder=True)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 3) Do the same for validation (but typically you skip shuffle for val):
val_ds = tf.data.Dataset.from_tensor_slices(
    (user_input_test, item_input_test, labels_test)
)
val_ds = (
    val_ds
    .batch(batch_size, drop_remainder=False)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 4) Now call .fit on the Datasets:
history_with_feats_2 = model_with_feats_2.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
    callbacks=[rank_cb],
    shuffle=False  # we already shuffled in the Dataset pipeline
)


Epoch 1/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step - auc: 0.8117 - loss: 0.4817 — epoch 1:   P@5=0.098, NDCG@5=0.105, MAP@5=0.056, ILD@5=0.308, Nov@5=0.578;  P@10=0.084, NDCG@10=0.093, MAP@10=0.036, ILD@10=0.305, Nov@10=0.608;  P@20=0.079, NDCG@20=0.134, MAP@20=0.048, ILD@20=0.303, Nov@20=0.658;
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 606s 245ms/step - auc: 0.8117 - loss: 0.4817 - val_auc: 0.9101 - val_loss: 0.3627 - Precision@5: 0.0980 - NDCG@5: 0.1053 - MAP@5: 0.0562 - ILD@5: 0.3079 - Novelty@5: 0.5781 - Precision@10: 0.0840 - NDCG@10: 0.0933 - MAP@10: 0.0365 - ILD@10: 0.3052 - Novelty@10: 0.6076 - Precision@20: 0.0785 - NDCG@20: 0.1340 - MAP@20: 0.0477 - ILD@20: 0.3028 - Novelty@20: 0.6582
Epoch 2/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - auc: 0.9062 - loss: 0.3616 — epoch 2:   P@5=0.112, NDCG@5=0.121, MAP@5=0.071, ILD@5=0.305, Nov@5=0.637;  P@10=0.109, NDCG@10=0.115, MAP@10=0.048, ILD@10=0.305, Nov@10=0.664;  P@20=0.095, NDCG@20=0.160, MAP@20=0.062, ILD@20=0.303, Nov@20=0.703;
24



# BERT + ResNet



In [61]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Embedding, Flatten, Multiply,
    Concatenate, Dense, Dropout
)
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision

# 0) Optional: mixed-precision
mixed_precision.set_global_policy("mixed_float16")

# 1) Hyperparameters
K_mf       = 16
K_mlp      = 32
mlp_layers = [64, 32]
dropout    = 0.2

# Suppose:
#   bert_embed_layer   = Embedding(num_items, D1,
#                                embeddings_initializer=Constant(bert_embeddings), trainable=False)
#   resnet_embed_layer = Embedding(num_items, D2,
#                                embeddings_initializer=Constant(portrait_embeddings), trainable=False)

# 2) Inputs
user_in = Input(shape=(), dtype=tf.int32, name="user_input")
item_in = Input(shape=(), dtype=tf.int32, name="item_input")

# 3) GMF branch
u_mf = Embedding(num_users, K_mf, name="u_embed_mf")(user_in)
i_mf = Embedding(num_items, K_mf, name="i_embed_mf")(item_in)
u_lat_mf = Flatten()(u_mf)
i_lat_mf = Flatten()(i_mf)
mf_vec   = Multiply()([u_lat_mf, i_lat_mf])

# 4) MLP branch — learned embeddings
u_mlp = Embedding(num_users, K_mlp, name="u_embed_mlp")(user_in)
i_mlp = Embedding(num_items, K_mlp, name="i_embed_mlp")(item_in)
u_lat_mlp = Flatten()(u_mlp)
i_lat_mlp = Flatten()(i_mlp)

# 5) Look up both feature sets
bert_feat   = bert_embed_layer(item_in)    # shape=(batch, D1)
resnet_feat = resnet_embed_layer(item_in)  # shape=(batch, D2)

# 6) (Optional) Project each to smaller dimension, e.g. 64
bert_proj   = Dense(64, activation="relu", name="proj_bert")(bert_feat)
resnet_proj = Dense(64, activation="relu", name="proj_resnet")(resnet_feat)

# 7) Concatenate all MLP inputs
mlp_input = Concatenate(name="mlp_input")([
    u_lat_mlp,
    i_lat_mlp,
    bert_proj,
    resnet_proj
])  # shape = (batch, 2*K_mlp + 64 + 64)

# 8) MLP layers
x = mlp_input
for idx, size in enumerate(mlp_layers):
    x = Dense(size, activation="relu", name=f"mlp_dense_{idx}")(x)
    x = Dropout(dropout)(x)

# 9) Final fusion + output
final = Concatenate()([mf_vec, x])
out   = Dense(1, activation="sigmoid", name="prediction")(final)

model_both = Model(inputs=[user_in, item_in], outputs=out)
model_both.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name="auc")]
)
model_both.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mlp         │ (None, 32)        │  1,690,272 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mlp         │ (None, 32)        │    137,184 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_embed_lookup   │ (None, 768)       │  3,292,416 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet_embed_lookup │ (None, 2048)      │  8,779,776 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_14          │ (None, 32)        │          0 │ u_embed_mlp[0][0] │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_15          │ (None, 32)        │          0 │ i_embed_mlp[0][0] │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ proj_bert (Dense)   │ (None, 64)        │     49,216 │ bert_embed_looku… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ proj_resnet (Dense) │ (None, 64)        │    131,136 │ resnet_embed_loo… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input           │ (None, 192)       │          0 │ flatten_14[0][0], │
│ (Concatenate)       │                   │            │ flatten_15[0][0], │
│                     │                   │            │ proj_bert[0][0],  │
│                     │                   │            │ proj_resnet[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_0 (Dense) │ (None, 64)        │     12,352 │ mlp_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mf          │ (None, 16)        │    845,136 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mf          │ (None, 16)        │     68,592 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 64)        │          0 │ mlp_dense_0[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_12          │ (None, 16)        │          0 │ u_embed_mf[0][0]  │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_13          │ (None, 16)        │          0 │ i_embed_mf[0][0]  │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 15,008,209 (57.25 MB)

 Trainable params: 2,936,017 (11.20 MB)

 Non-trainable params: 12,072,192 (46.05 MB)

Training

In [62]:
ks = [5, 10, 20]
rank_cb = MultiKRankingCallback(
    train_pos=train_pos,      # user → set of train-positive items
    test_pos=test_pos,        # user → set of test-positive items
    num_items=num_items,      # total number of distinct items (4287)
    Ks=ks,                     # compute Precision@k, NDCG@k, MAP@k
    verbose=1                 # set to 1 to print metrics each epoch
)

In [63]:
batch_size = 4096  # increase if your GPU/Colab can handle it (maybe 4096)
buffer_size = 100_000

# 1) Build a Dataset from your three 1D NumPy arrays
#    Each element is (u_idx, i_idx, label).
train_ds = tf.data.Dataset.from_tensor_slices((user_input, item_input, labels))

# 2) Shuffle → batch → prefetch.  Drop remainder so each batch is exactly `batch_size`.
train_ds = (
    train_ds
    .shuffle(buffer_size, seed=42)
    .batch(batch_size, drop_remainder=True)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 3) Do the same for validation (but typically you skip shuffle for val):
val_ds = tf.data.Dataset.from_tensor_slices(
    (user_input_test, item_input_test, labels_test)
)
val_ds = (
    val_ds
    .batch(batch_size, drop_remainder=False)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 4) Now call .fit on the Datasets:
history_both = model_both.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
    callbacks=[rank_cb],
    shuffle=False  # we already shuffled in the Dataset pipeline
)


Epoch 1/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - auc: 0.8329 - loss: 0.4591 — epoch 1:   P@5=0.108, NDCG@5=0.119, MAP@5=0.066, ILD@5=0.312, Nov@5=0.499;  P@10=0.091, NDCG@10=0.102, MAP@10=0.042, ILD@10=0.309, Nov@10=0.551;  P@20=0.083, NDCG@20=0.145, MAP@20=0.054, ILD@20=0.306, Nov@20=0.626;
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 638s 258ms/step - auc: 0.8329 - loss: 0.4591 - val_auc: 0.9138 - val_loss: 0.3591 - Precision@5: 0.1080 - NDCG@5: 0.1194 - MAP@5: 0.0659 - ILD@5: 0.3118 - Novelty@5: 0.4994 - Precision@10: 0.0910 - NDCG@10: 0.1025 - MAP@10: 0.0416 - ILD@10: 0.3094 - Novelty@10: 0.5511 - Precision@20: 0.0830 - NDCG@20: 0.1455 - MAP@20: 0.0544 - ILD@20: 0.3062 - Novelty@20: 0.6263
Epoch 2/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step - auc: 0.9135 - loss: 0.3488 — epoch 2:   P@5=0.116, NDCG@5=0.129, MAP@5=0.074, ILD@5=0.303, Nov@5=0.546;  P@10=0.111, NDCG@10=0.121, MAP@10=0.051, ILD@10=0.301, Nov@10=0.592;  P@20=0.099, NDCG@20=0.169, MAP@20=0.066, ILD@20=0.304, Nov@20=0.654;
24

# Ejemplos recomendaciones

In [101]:
# Elige el usuario (raw ID) para el que quieres recomendaciones
raw_user_id = list(user_interactions.keys())[0]  # o pon el que quieras
user_idx    = userid2idx[raw_user_id]


In [102]:
# Averigua que libros ya vio en train
seen_raw = user_interactions.get(raw_user_id, [])
seen_idx = { bookid2idx[b] for b in seen_raw if b in bookid2idx }

In [103]:
# Construye la lista de candidatos excluyendo los vistos
all_items  = np.arange(len(bookid2idx), dtype=np.int32)
candidates = np.setdiff1d(all_items, list(seen_idx), assume_unique=True)

## Vanilla Model

In [104]:
# Pide al modelo la puntuación de cada candidato
user_batch = np.full(candidates.shape, user_idx, dtype=np.int32)
scores     = model_vanilla.predict([user_batch, candidates], batch_size=4096).flatten()

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [105]:
# Selecciona los top-5
top_n      = 5
# find the positions of the top_n highest scores
top_pos   = np.argpartition(-scores, top_n-1)[:top_n]
# now sort those by actual score
ordering = np.argsort(-scores[top_pos])
top_items = candidates[top_pos][ordering]
top_scores = scores[top_pos][ordering]

In [106]:
# Mapea de vuelta a los raw book IDs y une con metadata
idx2bookid   = {v:k for k,v in bookid2idx.items()}
top_book_ids = [idx2bookid[i] for i in top_items]

In [107]:
top_df = pd.DataFrame({
    "book_id":    top_book_ids,
    "pred_score": top_scores,
})
top_df["rank"] = np.arange(1, len(top_df)+1)

In [108]:
# Merge with your books metadata
rec_df = top_df.merge(df_books, on="book_id", how="left")

In [109]:
# Re-order columns
rec_df = rec_df[["rank", "book_id", "pred_score"] +
                [c for c in df_books.columns if c!="book_id"]]

In [110]:
cols = ['rank', 'book_id', 'pred_score', 'book_desc', 'title', 'image_url']

In [111]:
# Muestra resultado
print(f"Top {top_n} recomendaciones para user {raw_user_id}:")
display(rec_df[cols])

Top 5 recomendaciones para user 1:


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,rank,book_id,pred_score,book_desc,title,image_url
0,1,5,0.979980,Alternate Cover Edition ISBN: 0743273567 (ISBN...,The Great Gatsby,https://images.gr-assets.com/books/1490528560m...
1,2,38,0.978027,"A funny, often poignant tale of boy meets girl...",The Time Traveler's Wife,https://images.gr-assets.com/books/1437728815m...
2,3,172,0.978027,Acclaimed by many as the world's greatest nove...,Anna Karenina,https://images.gr-assets.com/books/1352422904m...
3,4,118,0.977539,"In 1949, four Chinese women, recent immigrants...",The Joy Luck Club,https://images.gr-assets.com/books/1304978653m...
4,5,94,0.976562,"The brilliant, bestselling, landmark novel tha...",One Hundred Years of Solitude,https://images.gr-assets.com/books/1327881361m...


In [113]:
from IPython.display import HTML, display

def show_recs_with_images(df):
    html = "<table>"
    html += "<tr><th>Rank</th><th>Title</th><th>Cover</th></tr>"
    for _, r in df.iterrows():
        html += (
            f"<tr>"
            f"<td>{r['rank']}</td>"
            f"<td>{r['title']}</td>"
           # f"<td>{r['book_desc']}</td>"
            f"<td><img src='{r['image_url']}' width='80'></td>"
            f"</tr>"
        )
    html += "</table>"
    display(HTML(html))

show_recs_with_images(rec_df)

Rank,Title,Cover
1,The Great Gatsby,
2,The Time Traveler's Wife,
3,Anna Karenina,
4,The Joy Luck Club,
5,One Hundred Years of Solitude,


## NCF + BERT

In [114]:
# Pide al modelo la puntuación de cada candidato
user_batch = np.full(candidates.shape, user_idx, dtype=np.int32)
scores     = model_with_feats.predict([user_batch, candidates], batch_size=4096).flatten()

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step


In [115]:
# Selecciona los top-5
top_n      = 5
# find the positions of the top_n highest scores
top_pos   = np.argpartition(-scores, top_n-1)[:top_n]
# now sort those by actual score
ordering = np.argsort(-scores[top_pos])
top_items = candidates[top_pos][ordering]
top_scores = scores[top_pos][ordering]

In [116]:
# Mapea de vuelta a los raw book IDs y une con metadata
idx2bookid   = {v:k for k,v in bookid2idx.items()}
top_book_ids = [idx2bookid[i] for i in top_items]

In [117]:
top_df = pd.DataFrame({
    "book_id":    top_book_ids,
    "pred_score": top_scores,
})
top_df["rank"] = np.arange(1, len(top_df)+1)

In [118]:
# Merge with your books metadata
rec_df = top_df.merge(df_books, on="book_id", how="left")

In [119]:
# Re-order columns
rec_df = rec_df[["rank", "book_id", "pred_score"] +
                [c for c in df_books.columns if c!="book_id"]]

In [120]:
cols = ['rank', 'book_id', 'pred_score', 'book_desc', 'title', 'image_url']

In [121]:
# Muestra resultado
print(f"Top {top_n} recomendaciones para user {raw_user_id}:")
display(rec_df[cols])

Top 5 recomendaciones para user 1:


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,rank,book_id,pred_score,book_desc,title,image_url
0,1,30,0.991211,"On a warm summer morning in North Carthage, Mi...",Gone Girl,https://images.gr-assets.com/books/1339602131m...
1,2,5,0.990234,Alternate Cover Edition ISBN: 0743273567 (ISBN...,The Great Gatsby,https://images.gr-assets.com/books/1490528560m...
2,3,61,0.984863,Rachel catches the same commuter train every m...,The Girl on the Train,https://images.gr-assets.com/books/1490903702m...
3,4,146,0.983887,"It begins with a boy. Theo Decker, a thirteen-...",The Goldfinch,https://images.gr-assets.com/books/1451554970m...
4,5,94,0.976074,"The brilliant, bestselling, landmark novel tha...",One Hundred Years of Solitude,https://images.gr-assets.com/books/1327881361m...


In [122]:
from IPython.display import HTML, display

def show_recs_with_images(df):
    html = "<table>"
    html += "<tr><th>Rank</th><th>Title</th><th>Cover</th></tr>"
    for _, r in df.iterrows():
        html += (
            f"<tr>"
            f"<td>{r['rank']}</td>"
            f"<td>{r['title']}</td>"
           # f"<td>{r['book_desc']}</td>"
            f"<td><img src='{r['image_url']}' width='80'></td>"
            f"</tr>"
        )
    html += "</table>"
    display(HTML(html))

show_recs_with_images(rec_df)

Rank,Title,Cover
1,Gone Girl,
2,The Great Gatsby,
3,The Girl on the Train,
4,The Goldfinch,
5,One Hundred Years of Solitude,


## NCF + ResNet

In [125]:
# Pide al modelo la puntuación de cada candidato
user_batch = np.full(candidates.shape, user_idx, dtype=np.int32)
scores     = model_with_feats_2.predict([user_batch, candidates], batch_size=4096).flatten()

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [126]:
# Selecciona los top-5
top_n      = 5
# find the positions of the top_n highest scores
top_pos   = np.argpartition(-scores, top_n-1)[:top_n]
# now sort those by actual score
ordering = np.argsort(-scores[top_pos])
top_items = candidates[top_pos][ordering]
top_scores = scores[top_pos][ordering]

In [127]:
# Mapea de vuelta a los raw book IDs y une con metadata
idx2bookid   = {v:k for k,v in bookid2idx.items()}
top_book_ids = [idx2bookid[i] for i in top_items]

In [128]:
top_df = pd.DataFrame({
    "book_id":    top_book_ids,
    "pred_score": top_scores,
})
top_df["rank"] = np.arange(1, len(top_df)+1)

In [129]:
# Merge with your books metadata
rec_df = top_df.merge(df_books, on="book_id", how="left")

In [130]:
# Re-order columns
rec_df = rec_df[["rank", "book_id", "pred_score"] +
                [c for c in df_books.columns if c!="book_id"]]

In [131]:
cols = ['rank', 'book_id', 'pred_score', 'book_desc', 'title', 'image_url']

In [132]:
# Muestra resultado
print(f"Top {top_n} recomendaciones para user {raw_user_id}:")
display(rec_df[cols])

Top 5 recomendaciones para user 1:


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,rank,book_id,pred_score,book_desc,title,image_url
0,1,172,0.983887,Acclaimed by many as the world's greatest nove...,Anna Karenina,https://images.gr-assets.com/books/1352422904m...
1,2,146,0.979980,"It begins with a boy. Theo Decker, a thirteen-...",The Goldfinch,https://images.gr-assets.com/books/1451554970m...
2,3,5,0.974609,Alternate Cover Edition ISBN: 0743273567 (ISBN...,The Great Gatsby,https://images.gr-assets.com/books/1490528560m...
3,4,291,0.973145,An unforgettable journey into one man's remark...,Cutting for Stone,https://images.gr-assets.com/books/1327931601m...
4,5,820,0.971680,Librarian's note: See alternate cover edition ...,The Remains of the Day,https://images.gr-assets.com/books/1327128714m...


In [133]:
from IPython.display import HTML, display

def show_recs_with_images(df):
    html = "<table>"
    html += "<tr><th>Rank</th><th>Title</th><th>Cover</th></tr>"
    for _, r in df.iterrows():
        html += (
            f"<tr>"
            f"<td>{r['rank']}</td>"
            f"<td>{r['title']}</td>"
           # f"<td>{r['book_desc']}</td>"
            f"<td><img src='{r['image_url']}' width='80'></td>"
            f"</tr>"
        )
    html += "</table>"
    display(HTML(html))

show_recs_with_images(rec_df)

Rank,Title,Cover
1,Anna Karenina,
2,The Goldfinch,
3,The Great Gatsby,
4,Cutting for Stone,
5,The Remains of the Day,


## NCF + BERT + ResNet

In [139]:
# Pide al modelo la puntuación de cada candidato
user_batch = np.full(candidates.shape, user_idx, dtype=np.int32)
scores     = model_both.predict([user_batch, candidates], batch_size=4096).flatten()

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step 


In [140]:
# Selecciona los top-5
top_n      = 5
# find the positions of the top_n highest scores
top_pos   = np.argpartition(-scores, top_n-1)[:top_n]
# now sort those by actual score
ordering = np.argsort(-scores[top_pos])
top_items = candidates[top_pos][ordering]
top_scores = scores[top_pos][ordering]

In [141]:
# Mapea de vuelta a los raw book IDs y une con metadata
idx2bookid   = {v:k for k,v in bookid2idx.items()}
top_book_ids = [idx2bookid[i] for i in top_items]

In [142]:
top_df = pd.DataFrame({
    "book_id":    top_book_ids,
    "pred_score": top_scores,
})
top_df["rank"] = np.arange(1, len(top_df)+1)

In [143]:
# Merge with your books metadata
rec_df = top_df.merge(df_books, on="book_id", how="left")

In [144]:
# Re-order columns
rec_df = rec_df[["rank", "book_id", "pred_score"] +
                [c for c in df_books.columns if c!="book_id"]]

In [145]:
cols = ['rank', 'book_id', 'pred_score', 'book_desc', 'title', 'image_url']

In [146]:
# Muestra resultado
print(f"Top {top_n} recomendaciones para user {raw_user_id}:")
display(rec_df[cols])

Top 5 recomendaciones para user 1:


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,rank,book_id,pred_score,book_desc,title,image_url
0,1,30,0.995117,"On a warm summer morning in North Carthage, Mi...",Gone Girl,https://images.gr-assets.com/books/1339602131m...
1,2,61,0.994141,Rachel catches the same commuter train every m...,The Girl on the Train,https://images.gr-assets.com/books/1490903702m...
2,3,5,0.989258,Alternate Cover Edition ISBN: 0743273567 (ISBN...,The Great Gatsby,https://images.gr-assets.com/books/1490528560m...
3,4,146,0.986816,"It begins with a boy. Theo Decker, a thirteen-...",The Goldfinch,https://images.gr-assets.com/books/1451554970m...
4,5,8,0.982910,The hero-narrator of The Catcher in the Rye is...,The Catcher in the Rye,https://images.gr-assets.com/books/1398034300m...


In [147]:
from IPython.display import HTML, display

def show_recs_with_images(df):
    html = "<table>"
    html += "<tr><th>Rank</th><th>Title</th><th>Cover</th></tr>"
    for _, r in df.iterrows():
        html += (
            f"<tr>"
            f"<td>{r['rank']}</td>"
            f"<td>{r['title']}</td>"
          #  f"<td>{r['book_desc']}</td>"
            f"<td><img src='{r['image_url']}' width='80'></td>"
            f"</tr>"
        )
    html += "</table>"
    display(HTML(html))

show_recs_with_images(rec_df)

Rank,Title,Cover
1,Gone Girl,
2,The Girl on the Train,
3,The Great Gatsby,
4,The Goldfinch,
5,The Catcher in the Rye,


## Interacted items

In [150]:
from IPython.display import HTML, display

def show_seen_with_images(raw_user_id, user_interactions, books_df, userid2idx, bookid2idx, idx2bookid):
    # 1) Find the user idx and the raw IDs they’ve seen
    user_idx = userid2idx[raw_user_id]
    seen_raw = user_interactions.get(raw_user_id, [])
    seen_ids = [ bookid2idx[b] for b in seen_raw if b in bookid2idx ]

    # 2) Map back to raw book_id and grab the metadata
    seen_book_ids = [ idx2bookid[i] for i in seen_ids ]
    seen_df = books_df[books_df['book_id'].isin(seen_book_ids)].copy()
    # preserve original order
    seen_df = seen_df.set_index('book_id').loc[seen_book_ids].reset_index()

    # 3) Render HTML table with cover images
    html = "<h3>Already Interacted by User</h3>"
    html += "<table>"
    html += "<tr><th>Title</th><th>Cover</th></tr>"
    for _, r in seen_df.iterrows():
        title = r.get('title', r.get('original_title', ''))
        img   = r.get('image_url', '')
        html += (
            f"<tr>"
            f"<td>{title}</td>"
            f"<td><img src='{img}' width='60'></td>"
            f"</tr>"
        )
    html += "</table>"
    display(HTML(html))

# Usage:
idx2bookid = {v:k for k,v in bookid2idx.items()}
show_seen_with_images(
    raw_user_id,
    user_interactions,
    df_books,
    userid2idx,
    bookid2idx,
    idx2bookid
)


Title,Cover
"The Shadow of the Wind (The Cemetery of Forgotten Books, #1)",
Never Let Me Go,
Housekeeping,
"Gilead (Gilead, #1)",
The History of Love,
Balzac and the Little Chinese Seamstress,
West with the Night,
Reading Lolita in Tehran,
The Kite Runner,
The Stone Diaries,
